# Prepare (required)

In [ ]:
#@markdown ## Mount to drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
#@markdown ## Create Folder
#@markdown #### Warning: Don't do this step if you already setup a minecraft server in your drive
import os, os.path
if os.path.exists("/content/drive/MyDrive/minecraftServer"):
  print("Folder exists!")
else:
  print("Creating Folder...")
  os.system("mkdir /content/drive/MyDrive/minecraftServer")
  print("Done!")

if os.path.exists("/content/drive/MyDrive/minecraftServer/java"):
  print("Folder exists!")
else:
  print("Creating Folder...")
  os.system("mkdir /content/drive/MyDrive/minecraftServer/java")
  print("Done!")

if os.path.exists("/content/drive/MyDrive/minecraftServer/bedrock"):
  print("Folder exists!")
else:
  print("Creating Folder...")
  os.system("mkdir /content/drive/MyDrive/minecraftServer/bedrock")
  print("Done!")

In [2]:
#@markdown ## Install requirements
!pip install -U pyngrok
!pip install -U minecraftVersions

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 761.3/761.3 KB 19.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyngrok: filename=pyngrok-5.2.1-py3-none-any.whl size=19790 sha256=1f87fbb1aae8d57bff53d4e247278543245b99745fb97039a39eb7fa0474d9c4
  Stored in directory: /root/.cache/pip/wheels/f6/89/59/49d4249e00957e94813ac136a335d10ed2e09a856c5096f95c
Successfully built pyngrok
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#@markdown ## Ngrok auth token (For tunneling the server) [Required]
import getpass
print("Get auth token from https://dashboard.ngrok.com/get-started/your-authtoken")
ngtoken = getpass.getpass()
!echo $ngtoken >> /content/drive/MyDrive/minecraftServer/ngtoken.txt

In [ ]:
#@markdown ## Discord Webhook (For announcing server IP) [optional]
import getpass
discordWebhook = getpass.getpass()
!echo $discordWebhook >> /content/drive/MyDrive/minecraftServer/discordWebhook.txt

# For Minecraft Java Edition

## 1. Update Java

In [ ]:
#@markdown ## Java 16
%cd /content/
!wget https://files03.tchspt.com/temp/jdk-16.0.2_linux-x64_bin.deb
!sudo apt install /content/jdk-16.0.2_linux-x64_bin.deb
!rm -rf /content/jdk-16.0.2_linux-x64_bin.deb
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/jdk-16.0.2"
!update-alternatives --install /usr/bin/java java /usr/lib/jvm/jdk-16.0.2/bin/java 1919
!java -version

In [ ]:
#@markdown ## Java 17 (Paper 1.18 and above)
%cd /content/
!wget https://download.oracle.com/java/17/archive/jdk-17.0.6_linux-x64_bin.deb
!sudo apt install /content/jdk-17.0.6_linux-x64_bin.deb
!rm -rf /content/jdk-17.0.6_linux-x64_bin.deb
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/jdk-17"
!update-alternatives --install /usr/bin/java java /usr/lib/jvm/jdk-17/bin/java 1919
!java -version

In [ ]:
#@markdown ## Java 19 (Latest version)
%cd /content/
!wget https://download.oracle.com/java/19/latest/jdk-19_linux-x64_bin.deb
!sudo apt install /content/jdk-19_linux-x64_bin.deb
!rm -rf /content/jdk-19_linux-x64_bin.deb
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/jdk-19"
!update-alternatives --install /usr/bin/java java /usr/lib/jvm/jdk-19/bin/java 1919
!java -version

In [ ]:
#@markdown ## 2. Set config
#@markdown ### Run the cell to change the config
import ipywidgets as widgets
from IPython.display import display
import minecraftVersions as MCVersion
import requests

mc = MCVersion.MCVersion()
paperlist = requests.get("https://gist.githubusercontent.com/osipxd/6119732e30059241c2192c4a8d2218d9/raw/c2beaf5ee892c55f21ec66d5e1e1969ea59082e4/paper-versions.json")

javaServerDownloadUrl = mc.latest().release().server().url
paperVersion = paperlist.json()["latest"]
vanillaVersionGui = widgets.Dropdown(
    options=mc.getAllVersions(),
    value=mc.latest().release().version,
    description='MC Version:',
)
paperVersionGui = widgets.Dropdown(
    options=[i for i in paperlist.json()["versions"].keys()],
    value=paperlist.json()["latest"],
    description='PaperMC Version:',
)
saveButton = widgets.Button(description="Save")
def save_config(*arg):
  global vanillaVersionGui
  global javaServerDownloadUrl
  global paperVersion
  javaServerDownloadUrl = mc.getVersion(vanillaVersionGui.value).server().url
  paperVersion = paperVersionGui.value
  print("Saved")

saveButton.on_click(save_config)

display(vanillaVersionGui)
display(paperVersionGui)
display(saveButton)


## 3. Download server software (click the arrow to expand)
#### Warning: Don't do this step if you already setup a minecraft server in your drive

In [ ]:
#@markdown ## Vanilla

import os, os.path
if os.path.exists("/content/drive/MyDrive/minecraftServer/java/vanilla"):
  print("Folder exists!")
else:
  print("Creating Folder...")
  os.system("mkdir /content/drive/MyDrive/minecraftServer/java/vanilla")
  print("Downloading java server...")
  os.system(f"wget {javaServerDownloadUrl} -O /content/drive/MyDrive/minecraftServer/java/vanilla/server.jar")
  print("Done!")

In [ ]:
#@markdown ## PaperMC

import os, os.path
import json
import requests

res = requests.get("https://gist.githubusercontent.com/osipxd/6119732e30059241c2192c4a8d2218d9/raw/c2beaf5ee892c55f21ec66d5e1e1969ea59082e4/paper-versions.json")
paperVersionList = json.loads(res.text)
paperDownloadUrl = paperVersionList["versions"][paperVersion]
if os.path.exists("/content/drive/MyDrive/minecraftServer/java/paper"):
  print("Folder exists!")
else:
  print("Creating Folder...")
  os.system("mkdir /content/drive/MyDrive/minecraftServer/java/paper")
  print("Downloading java server...")
  os.system(f"wget {paperDownloadUrl} -O /content/drive/MyDrive/minecraftServer/java/paper/server.jar")
  print("Done!")

## 4. Start the server (Java)

In [ ]:
import pyngrok
from pyngrok import ngrok, conf
import getpass
import requests

%cd /content/

def getData(path):
  try:
    return open(path, "r").read().replace("\n", "")
  except:
    return ""
auth_token = getData("/content/drive/MyDrive/minecraftServer/ngtoken.txt")
webhookUrl = getData("/content/drive/MyDrive/minecraftServer/discordWebhook.txt")
if auth_token:
  !ngrok authtoken "$auth_token"
else:
  raise ValueError("Failed to get ngrok token!")

conf.get_default().region = "ap"
client = ngrok.connect(25565, 'tcp')
serverip = ((str(client).split('"')[1::2])[0]).replace('tcp://', '')
print("Server IP: " + serverip)
if webhookUrl:
  sendWebhook = requests.post(webhookUrl, json={
    "username": "Server IP",
    "content": "**Server IP:** `{}`".format(serverip)
  })
  print("Successfully sent the IP to the discord server!" if sendWebhook.status_code == 204 else "Failed to send the IP to the discord server! [{}]".format(sendWebhook.status_code))

software = "paper" #@param ["vanilla", "paper"] {type: "string"}
allocateRam = 11 #@param {type: "slider", min: 1, max: 11, step: 1}

%cd /content/drive/MyDrive/minecraftServer/java/"$software"
!echo "eula=true" >> /content/drive/MyDrive/minecraftServer/java/"$software"/eula.txt
!java -Xmx"$allocateRam"G -Xms"$allocateRam"G -jar /content/drive/MyDrive/minecraftServer/java/"$software"/server.jar nogui

# For Minecraft Bedrock Edition (Currently ngrok is not support UDP port so this will not work)

In [ ]:
#@markdown ## 1. Download bedrock server
#@markdown ### Current: 1.19.50.02
%cd /content/
bedrockServerDownloadUrl = "https://minecraft.azureedge.net/bin-linux/bedrock-server-1.19.50.02.zip" #@param {type: "string"}
!wget $bedrockServerDownloadUrl -O /content/bedrockServer.zip
!mkdir /content/drive/MyDrive/minecraftServer/bedrock/vanilla
!unzip bedrockServer.zip -d /content/drive/MyDrive/minecraftServer/bedrock/vanilla
!rm -rf /content/bedrockServer.zip



In [ ]:
#@markdown ## 2. Start The Server (Bedrock)
import pyngrok
from pyngrok import ngrok, conf
import getpass

print("Get auth token from https://dashboard.ngrok.com/get-started/your-authtoken")
auth_token = getpass.getpass()
!ngrok authtoken $auth_token

conf.get_default().region = "ap"
print(ngrok.connect(19132, 'udp'))

%cd /content/drive/MyDrive/minecraftServer/bedrock/vanilla
!LD_LIBRARY_PATH=. ./bedrock_server

# Tools

In [ ]:
#@markdown ## Quick add plugin (Paper)

import os, os.path

pluginUrl = "https://mediafilez.forgecdn.net/files/3957/782/SuperVanish-6.2.12.jar" #@param {type: "string"}

def downloadPlugin():
  global pluginUrl
  if pluginUrl:
    print("Fetching data from {}".format(pluginUrl))
    os.system(f"wget {pluginUrl} -P /content/drive/MyDrive/minecraftServer/java/paper/plugins/")
    print("Done! Saved to /content/drive/MyDrive/minecraftServer/java/paper/plugins")
  else:
    print("Invaild URL!")

if os.path.exists("/content/drive/MyDrive/minecraftServer/java/paper/plugins"):
  downloadPlugin()
else:
  os.system("mkdir /content/drive/MyDrive/minecraftServer/java/paper/plugins")
  downloadPlugin()
